## Import Libraries and Data

In [1]:
#import relevant libraries
import sys; sys.path
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge
#from sklearn.kernel_ridge import KernelRidge
#from sklearn.linear_model import Lasso
#from sklearn.svm import SVR
#from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
#from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import Pipeline

In [2]:
#load in functional and structural connectivity data + subj data file
#fc.csv and sc.csv are m8x matrices where m is number of subjects and n is number of features 
#(i.e. vectorised upper triangular)
#filepath is where the files are stored

fc = pd.read_csv('filepath/fc.csv', header=None).values
sc = pd.read_csv('filepath/sc.csv', header=None).values

#hc is just the concatenated sc and fc along the features axis
hc = np.concatenate((sc, fc), axis=1)

#subj_data.csv is phenotype data file 
T = pd.read_csv('filepath/subj_data.csv',header=0)

#extract subject IDs from subj_data table
subj = T.Subject.values

#extract the specific cognitive metrics from subj_data table
crystal = T.CogCrystalComp_AgeAdj.values
fluid = T.CogFluidComp_AgeAdj.values
total = T.CogTotalComp_AgeAdj.values

#concatenate cognitive metrics into single variable
cognition = ['Crystal','Fluid','Total']
#cognition = ['PMAT Correct', 'PMAT Response Time']
cog_metric = np.transpose(np.asarray([crystal, fluid, total]))
#cog_metric = np.transpose(np.asarray([pmat_cr, pmat_med]))

In [3]:
#remove rows with NaN for cognition
fc=fc[~np.isnan(cog_metric).any(axis=1)]
sc=sc[~np.isnan(cog_metric).any(axis=1)]
hc=hc[~np.isnan(cog_metric).any(axis=1)]
cog_metric = cog_metric[~np.isnan(cog_metric).any(axis=1)]

## Set Model Parameters

In [4]:
#set the number of permutations you want to perform
perm = 100
#set the number of cross-validation loops you want to perform
cv_loops = 5
#set the number of folds you want in the inner and outer folds of the nested cross-validation
k = 3
#set the proportion of data you want in your training set
train_size = .8
#set the number of variable you want to predict to be the number of variables stored in the cognition variablse
n_cog = np.size(cognition)

#set regression model type
regr = Ridge(normalize=True, max_iter=1000000)

#set hyperparameter grid space you want to search through for the model
alphas = np.linspace(100, 1000, num=10, endpoint=True, dtype=None, axis=0)

#set the param grid be to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#set x data to be the input variable you want to use
#ie fc, sc, or hc
X = fc

#set y to be the cognitive metrics you want to predict
Y = cog_metric

#set the number of features 
n_feat = X.shape[1]

In [5]:
#create arrays to store variables
#r^2 - coefficient of determination
r2 = np.zeros([perm,n_cog])
#explained variance
var = np.zeros([perm,n_cog])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([perm,n_cog])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([perm,n_cog])
#predictions made by the model
preds = np.zeros([perm,n_cog,int(np.ceil(X.shape[0]*(1-train_size)))])
#true test values for cognition
cogtest = np.zeros([perm,n_cog,int(np.ceil(X.shape[0]*(1-train_size)))])
#feature importance extracted from the model
featimp = np.zeros([perm,n_feat,n_cog])

In [6]:
#iterate through permutations
for p in range(perm):
    #print permutation # you're on
    print('Permutation %d' %(p+1))
    #split data into train and test sets
    x_train, x_test, cog_train, cog_test = train_test_split(X, Y, test_size=1-train_size, 
                                                            shuffle=True, random_state=p)

    
    #iterate through the cognitive metrics you want to predict
    for cog in range (n_cog):

        #print cognitive metrics being predicted 
        print ("Cognition: %s" % cognition[cog])
        
        #set y values for train and test based on     
        y_train = cog_train[:,cog]
        y_test = cog_test[:,cog]
        
        #store all the y_test values in a separate variable that can be accessed later if needed
        cogtest[p,cog,:] = y_test


        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
        nested_scores = []
        best_params = []
        

        #optimise regression model using nested CV
        print('Training Models')
        
        #go through the loops of the cross validation
        for i in range(cv_loops):


            #set parameters for inner and outer loops for CV
            inner_cv = KFold(n_splits=k, shuffle=True, random_state=i)
            outer_cv = KFold(n_splits=k, shuffle=True, random_state=i)
            
            #define regressor with grid-search CV for inner loop
            gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, 
                                      verbose=0, cv=inner_cv, scoring='r2')

            #fit regressor
            gridSearch.fit(x_train, y_train)

            #save parameters corresponding to the best score
            best_params.append(list(gridSearch.best_params_.values()))

            #call cross_val_score for outer loop
            nested_score = cross_val_score(gridSearch, X=x_train, y=y_train, cv=outer_cv, 
                                           scoring='r2', verbose=1)

            #record nested CV scores
            nested_scores.append(np.median(nested_score))

            #print how many cv loops are complete
            print("%d/%d Complete" % (i+1,cv_loops))
            
        #once all CV loops are complete, fit models based on optimised hyperparameters    
        print('Testing Models')


        #save optimised alpha values
        opt_alpha[p,cog] = np.median(best_params)


        #fit model using optimised hyperparameter
        model = Ridge(alpha = opt_alpha[p,cog], normalize=True, max_iter=1000000)
        model.fit(x_train, y_train);
        
        #compute r^2 (coefficient of determination)
        r2[p,cog]=model.score(x_test,y_test)

        #generate predictions from model
        preds[p,cog,:] = model.predict(x_test).ravel()
        
        #compute explained variance 
        var[p,cog] = explained_variance_score(y_test, preds[p,cog,:])

        #compute correlation between true and predicted
        corr[p,cog] = np.corrcoef(y_test, preds[p,cog,:])[1,0]

        #extract feature importance
        featimp[p,:,cog] = model.coef_


Permutation 1
Cognition: Crystal
Training Models


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.3s finished


1/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.1s finished


2/5 Complete


...

3/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   11.7s finished


4/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   11.0s finished


5/5 Complete
Testing Models


In [14]:
#save all of the outputs once everything is done
#filepath is where you want to store it

np.savetxt('filepath/r2.txt', r2, delimiter=',')
np.savetxt('filepath/var.txt', var, delimiter=',')
np.savetxt('filepath/corr.txt', corr, delimiter=',')
np.savetxt('filepath/alpha.txt', opt_alpha, delimiter=',')

np.savetxt('filepath/crystal_featimp.txt', featimp[:,:,0], delimiter=',')
np.savetxt('filepath/fluid_featimp.txt', featimp[:,:,1], delimiter=',')
np.savetxt('filepath/total_featimp.txt', featimp[:,:,2], delimiter=',')


np.savetxt('filepath/crystal_preds.txt', preds[:,0,:], delimiter=',')
np.savetxt('filepath/luid_preds.txt', preds[:,1,:], delimiter=',')
np.savetxt('filepath/total_preds.txt', preds[:,2,:], delimiter=',')

np.savetxt('filepath/crystal_cogtest.txt', cogtest[:,:,0], delimiter=',')
np.savetxt('filepath/fluid_cogtest.txt', cogtest[:,:,1], delimiter=',')
np.savetxt('filepath/total_cogtest.txt', cogtest[:,:,2], delimiter=',')
